In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import matplotlib.pyplot as plt

from dftorch.SCF import SCFx, SCFx_batch, scf_x_os
from dftorch.Spin import get_spin_energy, get_h_spin
from dftorch.Forces import Forces

from dftorch.Constants import Constants
from dftorch.Structure import Structure, StructureBatch
from dftorch.MD import MDXL, MDXLBatch
from dftorch.ESDriver import ESDriver, ESDriverBatch

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)

torch.cuda.empty_cache()

# MD with PBC and PME
### Note, PME does not work properly for small cells (< 20x20x20)
### Also, do not use PME without PBC.

## First, SCF and forces

In [2]:
%%time
dftorch_params = {
    'coul_method': 'PME', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.04,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-5,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 50,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#filename = '8RNT.xyz'      
filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  magnetic_hubbard_ldep=False
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=500.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-C.skf
### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 4.970438028, dEc = 168.921345355, t = 0.3 s

Iter 2
Res = 4.600868459, dEc = 3.820100542, t = 0.1 s

Iter 3
Res = 4.253910080, dEc = 3.337667615, t = 0.1 s

Iter 4
Res = 3.924703779, dEc = 2.920118794, t = 0.1 s

Iter 5
Res = 3.610067822, dEc = 2.556168261, t = 0.0 s

Iter 6
Res = 3.307865716, dEc = 2.238126024, t = 0.0 s

Iter 7
Res = 3.016962430, dEc = 1.960234594, t = 0.0 s

Iter 8
Res = 2.737416648, dEc = 1.717850650, t = 0.0 s

Iter 9
Res = 2.470613421, dEc = 1.506998633, t = 0.0 s

Iter 10
Res = 2.219054127, dEc = 1.324142228, t = 0.0 s

Iter 11
Res = 1.985715603, dEc = 1.166092188, t = 0.0 s

Iter 12
Res = 1.773235023, dEc = 1.0

In [3]:
#charges:
structure1.q

tensor([-0.2438, -0.7706, -0.4516, -0.3950, -0.7243,  0.2517, -0.4235, -0.4312,
        -0.3921, -0.4288,  0.2427, -0.3826, -0.4541, -0.4357, -0.4114, -0.2073,
        -0.3938, -0.7813, -0.7546, -0.4127])

In [4]:
#MO coeffs
structure1.Q

tensor([[ 1.7793e-01, -3.5631e-01,  4.9032e-01,  ..., -2.9076e-01,
          3.5272e-01, -2.0136e-01],
        [-6.8824e-02,  8.9261e-02, -1.3560e-02,  ..., -1.9987e-01,
          8.2232e-02,  1.5840e-02],
        [-2.2904e-04,  1.4454e-03, -2.4637e-04,  ..., -8.8955e-02,
         -2.0499e-02,  5.5811e-02],
        ...,
        [-3.3998e-02, -8.0351e-02, -1.3895e-01,  ..., -1.3726e-01,
         -1.1252e-01, -6.3736e-02],
        [-3.9725e-02, -9.2969e-02, -1.5810e-01,  ..., -2.6449e-01,
         -2.3171e-01, -1.4924e-01],
        [-1.4647e-01, -2.3625e-01, -2.1485e-01,  ..., -2.5871e-01,
         -3.5329e-01, -3.0009e-01]])

In [5]:
print("Total Energy: ", structure1.e_tot)
print("Chemical Potential: ", structure1.mu0)

Total Energy:  tensor(-138.3300)
Chemical Potential:  tensor(-44.0957)


## MD

In [6]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=100, dt=0.3, dump_interval=5, traj_filename='md_trj.xyz')

########## Step = 0 ##########
H0: 0.022 s
H1: 0.081 s
  rank: 0, Fel = 0.000164
  rank: 1, Fel = 0.000130
  rank: 2, Fel = 0.000019
KER: 0.219 s
F AND E: 0.001 s
ETOT = -138.32770692, EPOT = -138.54762254, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.3 s
0.0 GB


########## Step = 1 ##########
H0: 0.020 s
H1: 0.010 s
  rank: 0, Fel = 0.000355
  rank: 1, Fel = 0.000250
  rank: 2, Fel = 0.000024
KER: 0.018 s
F AND E: 0.001 s
ETOT = -138.32879379, EPOT = -139.16758910, EKIN = 0.21882875, T = 84.64663830, ResErr = 0.001296, t = 0.0 s
0.0 GB


########## Step = 2 ##########
H0: 0.019 s
H1: 0.013 s
  rank: 0, Fel = 0.000228
  rank: 1, Fel = 0.000190
  rank: 2, Fel = 0.000013
KER: 0.017 s
F AND E: 0.001 s
ETOT = -138.33023131, EPOT = -140.13703110, EKIN = 0.83735779, T = 323.90406995, ResErr = 0.002632, t = 0.1 s
0.0 GB


########## Step = 3 ##########
H0: 0.019 s
H1: 0.078 s
  rank: 0, Fel = 0.000032
KER: 0.048 s
F AND E: 0.001 s
ETOT = -138.33158250, EPOT = -141.38006778, EK

# MD without PBC and with full Coulomb
### Do not use PME without PBC.
### Turn on PBC if necessary

## First, SCF and forces

In [7]:
%%time
dftorch_params = {
    'coul_method': 'FULL', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.05,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-5,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 50,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#filename = '8RNT.xyz'      
filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
#LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
LBox = None
# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  magnetic_hubbard_ldep=False
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=500.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-C.skf
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
  Coulomb_k t 0.0 s

### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 5.363231945, dEc = 87.673319089, t = 0.0 s

Iter 2
Res = 4.657356940, dEc = 3.964600651, t = 0.0 s

Iter 3
Res = 3.986181824, dEc = 3.021792355, t = 0.0 s

Iter 4
Res = 3.350619078, dEc = 2.284765686, t = 0.0 s

Iter 5
Res = 2.760847345, dEc = 1.723482967, t = 0.0 s

Iter 6
Res = 2.237277013, dEc = 1.310032985, t = 0.0 s

Iter 7
Res = 1.798117539, dEc = 1.012956841, t = 0.0 s

Iter 8
Res = 1.451655050, dEc = 0.800232326, t = 0.0 s

Iter 9
Res = 1.201693635, dEc = 0.645979078, t = 0.0 s

Iter 10
Res = 1.040285109, dEc = 0.532373532, t = 0.0 s

Iter 11
Res = 0.935851488

In [8]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=100, dt=0.3, dump_interval=5, traj_filename='md_trj.xyz')

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
  Coulomb_k t 0.0 s

H0: 0.100 s
H1: 0.001 s
  rank: 0, Fel = 0.000163
  rank: 1, Fel = 0.000087
KER: 0.001 s
F AND E: 0.022 s
ETOT = -207.33150334, EPOT = -207.47284758, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.1 s
0.0 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
  Coulomb_k t 0.0 s

H0: 0.020 s
H1: 0.001 s
  rank: 0, Fel = 0.000382
  rank: 1, Fel = 0.000137
  rank: 2, Fel = 0.000045
KER: 0.001 s
F AND E: 0.001 s
ETOT = -207.33232463, EPOT = -207.86291058, EKIN = 0.14052295, T = 54.35663959, ResErr = 0.001302, t = 0.0 s
0.0 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
  Coulomb_k t 0.0 s

H0: 0.020 s
H1: 0.001 s
  rank: 0, Fel = 0.000259
  rank: 1, Fel = 0.000183
  rank: 2, Fel = 0.000028
KER: 0.001 s
F AND E: 0.001 s
ETOT = -207.33361092, EPOT = -208.47123406, EKIN = 0.52929966, T = 204.74200725, ResErr = 0.002

# Open-Shell single point. No MD for open-shells for now.
### PBC ok. PME not ok.
### Turn on PBC if necessary

## First, SCF and forces

In [9]:
%%time
dftorch_params = {
    'coul_method': 'FULL', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.05,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 50,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#filename = '8RNT.xyz'      
filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
#LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
LBox = None
# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                    'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
                    magnetic_hubbard_ldep=False
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=100.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=False)
#es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-C.skf
torch.float64 torch.int64
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
  Coulomb_k t 0.0 s

CPU times: total: 234 ms
Wall time: 241 ms


In [10]:
# structure1.H, structure.Hcoul, structure1.Hdipole, structure1.KK, structure1.D, \
# structure1.q, structure1.f, \
# structure1.mu0, structure1.e_coul_tmp, structure1.f_coul, structure1.dq_p1 = \
H, Hcoul, Hdipole, KK, D, Q, q_spin_atom, q_tot_atom, q_spin_sr, net_spin_sr, f, mu0, Ecoul, forces1, dq_p1 = \
    scf_x_os(structure1.el_per_shell, structure1.shell_types, structure1.n_shells_per_atom, const.shell_dim, const.w,
    dftorch_params, structure1.RX, structure1.RY, structure1.RZ, structure1.lattice_vecs,
    structure1.Nats, structure1.Nocc, structure1.n_orbitals_per_atom, structure1.Znuc, structure1.TYPE,
    structure1.Te,
    structure1.Hubbard_U,
    structure1.D0,
    structure1.H0, structure1.S, structure1.Z, structure1.e_field, structure1.C, structure1.req_grad_xyz)

### Do SCF ###
  Initial DM_Fermi
tensor(-8.1936) tensor([-8.1936, -8.1936])
tensor([-0.7075,  0.3191, -0.2453, -0.4050, -0.3318, -0.1537, -0.6987, -0.2105,
        -0.3484, -0.4840, -0.4302, -0.4960, -0.7002, -0.1962, -0.3061, -0.4868,
        -0.4483, -0.5056, -0.7045,  0.3545, -0.1923, -0.2198, -0.2645, -0.4580]) tensor(-8.3200)

Starting cycle
Iter 1
Res = 2.827265197, dEc = 90.003790309, t = 0.0 s

Iter 2
Res = 2.488659623, dEc = 3.112643023, t = 0.0 s

Iter 3
Res = 2.171777101, dEc = 2.466058098, t = 0.0 s

Iter 4
Res = 1.871681299, dEc = 1.948145140, t = 0.0 s

Iter 5
Res = 1.587503137, dEc = 1.538312447, t = 0.0 s

Iter 6
Res = 1.323636407, dEc = 1.220249551, t = 0.0 s

Iter 7
Res = 1.086650155, dEc = 0.978091498, t = 0.0 s

Iter 8
Res = 0.880981545, dEc = 0.795338407, t = 0.0 s

Iter 9
Res = 0.709177113, dEc = 0.656859218, t = 0.0 s

Iter 10
Res = 0.574582263, dEc = 0.550972725, t = 0.0 s

Iter 11
Res = 0.478742014, dEc = 0.469517782, t = 0.0 s

Iter 12
Res = 0.415420444, dEc 

### Open-shell forces

In [11]:
H_spin = get_h_spin(structure1.TYPE, net_spin_sr, const.w, structure1.n_shells_per_atom, structure1.shell_types)
H_spin = 0.5 * structure1.S * H_spin.unsqueeze(0).expand(2, -1, -1) * torch.tensor([[[1]],[[-1]]], device=H_spin.device)

In [12]:
Ftot, Fcoul, Fband0, Fdipole, FPulay, FScoul, FSdipole, Frep = Forces(H, structure1.Z, structure1.C,
                        D, structure1.D0,
                        structure1.dH0, structure1.dS, structure1.dCC, structure1.dVr,
                        structure1.e_field, structure1.Hubbard_U, q_tot_atom,
                        structure1.RX, structure1.RY, structure1.RZ,
                        structure1.Nats, const, structure1.TYPE)

In [13]:
net_spin = q_spin_atom[0] - q_spin_atom[1]

atom_ids = torch.repeat_interleave(torch.arange(len(structure1.n_orbitals_per_atom), device=structure1.H0.device), structure1.n_orbitals_per_atom) # Generate atom index for each orbital

FSspinA = torch.zeros((3, structure1.Nats), dtype=D.dtype, device=D.device)
factor = (net_spin * const.w[structure1.TYPE])*2
tmp = ((torch.tensor([[[1]],[[-1]]], device=H_spin.device) * D).unsqueeze(1)*structure1.dS.unsqueeze(0)).sum(0)
dS_times_D = tmp*factor[atom_ids].unsqueeze(-1)
dDS_XYZ_row_sum = torch.sum(dS_times_D, dim = 2) # sum of elements in each row
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), dDS_XYZ_row_sum) # sums elements from DS into q based on number of AOs, e.g. x4 p orbs for carbon or x1 for hydrogen
dDS_XYZ_col_sum = torch.sum(dS_times_D, dim=1)
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), -dDS_XYZ_col_sum);


In [14]:
Ftot = (Fband0 + FPulay + Fcoul + FScoul + (FSspinA)/2 + Frep)